# Análisis de clientes bancarios

Esta es la segunda versión del modelo **RNA** usando **K-folds cross validation**. 

## 1. Sesgo y varianza
**Sesgo:** El sesgo se define como la diferencia entre el valor real y, y el valor calculado y_hat en el **entrenamiento**. Esta medida esta inversamente relacionada con la complejidad de los modelos. A mayor complejidad del modelo, obtendremos un bajo sesgo. En el caso de redes neuronales aumentar la complejidad esta relasionado con aumentar el número de capas y la cantidad de neuronas dentro de cada capa. Se debe recordar que a medida que aumentamos la complejidad incurriremos en overfitting. Si lo analisas de manera mas consiente tiene logica, pues al sobre entrenar el modelo, las predicciones seran exactamente o casi las mismas medidas reales. Una condición no deseada. 

![Overfit and underfit](./img/Overfit.bmp)

**Varianza:** Por otro lado la varianza se trata de la variabilidad que obtenemos en los resultados del modelo al entregarle datos no vistos o mejor conocidos como datos de **test**. Es natural que las medidas obtenidas no sean exactamente iguales con datos no vistos por el modelo, pero se espera que el modelo sea capaz de encontrar patrones que le ayuden a interpretar estos nuevos datos, entregando resulatados diferencias minimas entre el resultado real y el predicho, osea con baja varianza. Nuevamente se puede inferir que a medida que un modelo presenta un overfit o sobre entrenamiento, obtengamos datos con una varianz alta. Esto se debe a que el modelo ha memorizado los datos de entrenamiento y no sabe como responder a datos no vistos. 


**Conclusión 1:** Podemos ver entoces que si disminuimos la varianza del modelo, terminaremos tambien por minimizar el sesgo. Nuestro objetivo entonces es diminuir la varianza de nuestros modelos. 

![Overfit and underfit](./img/VarianzaYSesgo.png)

## 2. k-fold cross validation
Es un metodo que nos ayuda a disminuir la varianza de nuesto sistema sin incurrir en overfiting. El metodo consiste en:

1. Dividir los datos en folds o grupos, se recomienda usar entre 5-10 divisiones. 
2. En cada iteracion tomaremos un grupo de datos para usarlo como test. 
3. En cada iteración se cambia el grupo de test de tal modo que el anterior grupo que se uso para test pasara a ser de entrenamiento.
4. De esta manera, al terminar de recorrer todas las iteraciones habremos usado todos los datos para test y para entrenamiento.
5. Al final el algoritmo nos entregara la presición media entre todos los k-folds que implementamos. Es natural que se haga así, pues si tomamos el mejor resultado, tendriamos un sesgo en la información.

De esta manera el sesgo o bias y la varianza tienden a disminuir sin caer en overfit. 

![Overfit and underfit](./img/k-fold.png)

Como se ha mensionado en el resumen del metódo, es necesario cambiar la manera en que se dividin los datos de train. Se procedera a cambiar la versión anterior del código para implementar K-folds, esto se hace en el tramo de código que implementa el modelo, todos los procesos anteriores son validos hasta este punto

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Cargar los datos
data = pd.read_csv("Churn_Modelling.csv")
print(data.shape)
data.head()

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#Separar variables dependientes e independientes
X = data.iloc[:,3:13].values
y = data.iloc[:,13].values
print(f'X = {X.shape} y= {y.shape}')

X = (10000, 10) y= (10000,)


In [4]:
#Decodificar datos categoricos
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Hacemos una variable dummy para la columna 1 de paises
ct = ColumnTransformer([("Geography", OneHotEncoder(), [1])], remainder = 'passthrough')
X = ct.fit_transform(X)

# Y una variable binaria o de nivel para la columna 2 genero de los clientes
labelencoder_X = LabelEncoder()
X[:, 4] = labelencoder_X.fit_transform(X[:, 4])

In [5]:
#Eliminar una columna de variables dummy para evitar la multicolinealidad
X = X[:,1:]
print(X.shape)

(10000, 11)


In [6]:
#Dividir datos en train y test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
print(f'X_train {X_train.shape}, X_test{X_test.shape}, y_train{y_train.shape}, y_test{y_test.shape}')

X_train (8000, 11), X_test(2000, 11), y_train(8000,), y_test(2000,)


In [7]:
#Normalizar los datos de train
from sklearn.preprocessing import StandardScaler
#Creamos un objeto base para escalar las variables
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
#Usamos la misma base de escala para los datos de test pues pertenecen al mismo grupo
X_test = scaler.fit_transform(X_test)

## Librerias
1. Se debe notar que usaremos wrappers o fragmentos de código de scikit_learn pero implementados en keras, osea es una combinación de las dos librerias. 
2. cross_val_score implementa K-folds y sera el encargado de ejecutar el entrenamiento de los datos. 
3. Es necesario definir nuestro modelo como una función a la cual llamaremos para implementar el nuevo metódo de train y test.
4. El encargado de hacer este proceso es el

In [8]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [9]:
#Construir la red neuronal artificial RNA
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dropout

In [10]:
#Función para construir nuestro modelo
def model_clasificator():
    model = Sequential()
    model.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu", input_dim = 11))
    model.add(Dense(units=6, kernel_initializer = 'uniform', activation='relu'))
    #Agregar la capa de salida
    model.add(Dense(units=1, kernel_initializer = 'uniform', activation='sigmoid'))
    #Mostrar un resumen del modelo 
    model.summary()
    #Compilar el modelo
    model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

 ***KerasClassifier*** requiere como argumentos:
- build_fn = función con nuestro modelo compilado
- Parametros de sklearn: Como se indico es un wrapper, el cual contiene la libreria de sklearn, por lo tanto, usaremos los parametros del .fit de esta libreria, epochs, batch_size. 

In [11]:
#Invocaremos el modelo mediante una variable clasifier
classifier = KerasClassifier(build_fn = model_clasificator, epochs = 100, batch_size=10)

 ***cross_val_score*** este proceso retorna varias medidas de los resultados de nuestro modelo, en este caso [accuracy] de cada iteración, por lo tanto es necesario guardar esto datos en un array. Los argumentos para esta función son:
1. estimator = objeto que usaremos para medir las presiciones obtenidos en este caso el clasifier
2. X = Conjunto de datos original que usaremos para entrenamiento del cual sacaremos un porcentaje para validar en cada iteración. 
3. y = Conjunto de datos con etiquetas de train
4. cv = número de folds para hacer la cross validation se recomienda 5 - 10. Se pueden superar este valor de 10 si tienes una maquina potente. 
5. n_jobs = Si deseas usar todos los core de tu máquina para resolver el ajuste de modelos pues escribir -1 en este parametro.
6. verbose = 1 para motrar los resultados de cada ajuste 0 para ocultarlos. 

In [12]:
accurasies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1, verbose = 1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:  1.9min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  2.6min finished


In [13]:
#Evaluar las presiciones obtenidas con el metodo de validación cruzada
accurasies

array([0.85750002, 0.83375001, 0.87374997, 0.82499999, 0.8725    ,
       0.86374998, 0.84125   , 0.81875002, 0.815     , 0.84249997])

Mediremos cual es la media de la presición obtenida y la varainza, de esta manera podemos determinar si son altas o bajas y podremos situarnos en alguna de las cuatro posibilidades vistas en el apartado inicial del código. 

In [14]:
media = accurasies.mean()
varianza = accurasies.std()
print(f'Varianza = {varianza} presición media = {media}')

Varianza = 0.020549096505934474 presición media = 0.8443749964237213


### Análisis
- Tenemos una varianza baja pero una presición un poco alejada del centro, por lo tanto podemos concluir que tenemos poca varianza y alto sesgo. 
- De la tabla de accuracys podemos ver qua las variaciones obtenidas entre los diferentes entrenamientos los porcentajes no difieren en +/- 2%. Si los valores obtenidos fuesen demasido grandes entre unos y otros, seria necesario añadir a nuestro modelo un metodo de regularización.
- Los metodos de regularización son metódos que permiten penalizar mas los datos demasiado alejados de los valores reales. 
- Uno de ellos es el dropout, el cual 'apaga' un porcentaje de neuronas por cada capa para evitar que todas las neuronas aprendan los mismos patrones, de esta manera se evita el overfit. 
- El dropout se puede implementar como uno de los metodos de la libreria layers de Keras. La linea que indica cual es la cantidad de neuronas a apagar se implementa inmediatamente despues de la capa donde apagaremos dichas neuronas. Solo requiere que indiquemos el porcentaje de las neuronas que deseamos apagar en cada entrenamiento.

## Mejorando la red - Grid Search
Hasta este punto los hiperparametros como: epochs, layers, neuronas por capa, batch_size, loss, optimizer entre otros se han mantenido fijos. Para mejorar los resultados entregados por la red neuronal es necesario variar estos valores. Para no tener que cambiar estos valores a ensayo y error se puede aplicar un metódo denominado grid search. 

**GridSearchCV:** consiste en fabricar una especie de red, en la que indicamos los posibles valores que deseamos proponer para evaluar en cada hiperparametro. Luego se hace el entrenamiento de nuestra red usando las diferentes combinaciones de valores que tenemos como posibles variaciones. Este metódo implementa el cross validation, de tal manera que usaremos el mismo metódo de funcionamiento. La rutina consta de:

1. Especificar el KerasClassifier entregandole como función el modelo que invoca nuestra estructura de red neuronal. Recordar que esta función especifica los argumentos de entrenamiento .fit  
2. Los hiper parametros de entrenamiento no los pasaremos como argumentos en el kerasClassifier. Es en este punto donde debemos implementar el grid-search para determinar que combinación es la que mejor resultado nos entrega. 
3. Como se mensiono debemos crear diferentes valores de los hiperparamentros, para crear este conjunto de datos usamos un diccionario, donde los key seran los nombres de los hiperparamentros y los valores serán sus posibles datos. 
5. Para implementar el grid usaremos la función GridSearchCV, los argumentos necesarios son:
- estimator = classifier, que sera el  KerasClassifier
- param_grid = parameters, diccionario con posibles valores para alimentar el grid-search. Se debe cuidar no tener demasiados hiperparametros, pues aumentarian el costo computacional. 
- scoring = 'accuracy',  es el parametro que deseamos clasificar
- cv = 10, es la cantidad de folds que usarmenos
6. Para modificar patrones dentro del modelo de red neuronal, debemos agregar como parametro de la función los hiperparametros que deseamos variar mediante el grid search.  

In [15]:
from sklearn.model_selection import GridSearchCV

Creamos la función de red neuronal con el parametro optimizer para variar el optimizador. En este caso: adam y rmsprop. 

rmsprop es un optimizador común para clasificación en redes neuronale de aprendizaje con pocas capas. 

In [22]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu", input_dim = 11))
    classifier.add(Dropout(0.1)) #Regularización de las neuronas
    classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))
    classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))
    classifier.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])
    print(f'{optimizer}')
    return classifier

Creamos la función que nos ayuda a llamar la función modelo en formato wraper. Osea envolviento las librerias de sklearn con keras. Como se ve solo se llama la función sin especificar los hiperparametros de .fit, estos hiperparametros los variaremos mediante el grid search.

In [23]:
classifier_b = KerasClassifier(build_fn = build_classifier)

Por facilidad declararemos los hiperparametros del grid search mediante un diccionario, para luedo entregarlos como parametro de la función GridSearchCV. 

In [24]:
parameters = {
    'batch_size' : [10, 25],
    'nb_epoch' : [150, 500], 
    'optimizer' : ['adam', 'rmsprop']
}

Finalmente se creara una variable grid_search que guardara el valor de la mejor estimación encontrada al intercambiar los diferentes hiperparametros del grid. La función GridSearchCV recibe los datos:
- Estimator = classifier en este caso es nuestro modelo invocado en un grapper de keras y sklearn
- param_grid = Diccionario con los hiperparametros de nuestra grid
- scoring = Es el valor que objetivo a minimizar mediante nuestro modelo en esta caso la presición.
- cv = Es el numero de folds que aplicaremos en el cross validation generalmente un valor entre 5 y 10.

In [25]:
grid_search = GridSearchCV(estimator = classifier_b, 
                           param_grid = parameters, 
                           scoring = 'accuracy', 
                           cv = 10)

Finalmente vamos a entrenar la red mediante el grid_search.fit y entregamos como es de esperar los datos de entrenamiento de variable independiente y dependiente. 

Este entrenmiento nos entrega los mejores parametros y la mejor variable objetivo lograda en este caso la presición accuracy. 

Estos datos se acceden mediante los metodos best_params_ y best_score_

In [26]:
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

adam
720/720 [==============================] - 0s 524us/step - loss: 0.5023 - accuracy: 0.7965
adam
720/720 [==============================] - 0s 547us/step - loss: 0.4979 - accuracy: 0.7958
adam
720/720 [==============================] - 0s 471us/step - loss: 0.4943 - accuracy: 0.7940
adam
720/720 [==============================] - 0s 497us/step - loss: 0.4901 - accuracy: 0.7964
adam
720/720 [==============================] - 0s 480us/step - loss: 0.5044 - accuracy: 0.7937
adam
720/720 [==============================] - 0s 481us/step - loss: 0.4914 - accuracy: 0.7943
adam
720/720 [==============================] - 0s 487us/step - loss: 0.5002 - accuracy: 0.7968
adam
720/720 [==============================] - 0s 504us/step - loss: 0.4945 - accuracy: 0.7960
adam
720/720 [==============================] - 0s 493us/step - loss: 0.4832 - accuracy: 0.7954
adam
720/720 [==============================] - 0s 508us/step - loss: 0.5021 - accuracy: 0.7956
rmsprop
720/720 [=======================

288/288 [==============================] - 0s 560us/step - loss: 0.5681 - accuracy: 0.7964
rmsprop
288/288 [==============================] - 0s 518us/step - loss: 0.6296 - accuracy: 0.7958
rmsprop
288/288 [==============================] - 0s 509us/step - loss: 0.5770 - accuracy: 0.7950
rmsprop
288/288 [==============================] - 0s 496us/step - loss: 0.5964 - accuracy: 0.7940
adam
800/800 [==============================] - 0s 485us/step - loss: 0.4891 - accuracy: 0.7955


In [27]:
print(f'Mejores parametros {best_parameters}')
print(f'Mejor presición accuracy obtenida {best_accuracy}')

Mejores parametros {'batch_size': 10, 'nb_epoch': 150, 'optimizer': 'adam'}
Mejor presición accuracy obtenida 0.796625
